In [1]:
import pandas as pd

In [2]:
df_patients = pd.read_parquet('../../ICU/tables/patients.parquet')

# Display the DataFrame
df_patients

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000100,Male,70,2019,2017 - 2019,None
1,10000562,Female,18,2016,2014 - 2016,None
2,10000671,Male,28,2019,2017 - 2019,None
3,10000683,Male,34,2015,2014 - 2016,None
4,10000788,Female,83,2017,2017 - 2019,None
...,...,...,...,...,...,...
492104,99999419,Male,47,2015,2014 - 2016,None
492105,99999577,Male,60,2019,2017 - 2019,None
492106,99999731,Male,38,2009,2008 - 2010,None
492107,99999856,Female,69,2019,2017 - 2019,None


In [3]:
df_patients=df_patients.drop(["anchor_year_group"], axis=1)

In [4]:
df_patients=df_patients.drop_duplicates()

In [5]:
df_patients

,subject_id,gender,anchor_age,anchor_year,dod
0,10000100,Male,70,2019,None
1,10000562,Female,18,2016,None
2,10000671,Male,28,2019,None
3,10000683,Male,34,2015,None
4,10000788,Female,83,2017,None
...,...,...,...,...,...
492104,99999419,Male,47,2015,None
492105,99999577,Male,60,2019,None
492106,99999731,Male,38,2009,None
492107,99999856,Female,69,2019,None


In [6]:
# dict_1=pd.Series(df_patients.gender.values,index = df_patients.subject_id).to_dict()
# dict_2=pd.Series(df_patients.anchor_age.values,index = df_patients.subject_id).to_dict()
# dict_3=pd.Series(df_patients.anchor_year.values,index = df_patients.subject_id).to_dict()
# dict_4=pd.Series(df_patients.dod.values,index = df_patients.subject_id).to_dict()

## Moving to admissions table

In [7]:
# clean admissions table

def clean_admissions_table():
    """
    exapmle usage:
    
    df = clean_admissions_table()
    """

    admissions_df = pd.read_parquet('../../ICU/tables/admissions.parquet')
    
    # insurance cleanup 
    ins_cleaning_dict = {
        'MEDICARE A': 'MEDICARE',
        'MEDICARE B': 'MEDICARE',
        'MEDICARE/MGD CARE': 'MEDICARE ADVANTAGE',
        'MEDICARE PART B ONLY': 'MEDICARE ',
        'MEDICARE': 'MEDICARE',
        'MEDICARE B/MCD': 'MEDICARE_MEDICAID',
        'MEDICARE B/MEDICAID': 'MEDICARE_MEDICAID',
        'MEDICARE/MEDICAID': 'MEDICARE_MEDICAID',
        'MGD CARE/MEDICARE': 'MEDICARE ADVANTAGE',
        'MEDICARE/COMMERCIAL': 'MEDICARE ADVANTAGE',
        'MEDICARE B/COMM': 'MEDICARE ADVANTAGE',
        'MEDICARE/BLUE CROSS':'MEDICARE ADVANTAGE',
        'MEDICARE B/BCBS': 'MEDICARE ADVANTAGE',
        'MEDICARE B/MGD CARE': 'MEDICARE ADVANTAGE',
        'MEDICARE/CHAMPUS': 'MEDICARE',
        'MEDICARE B/MISC': 'MEDICARE',
        'MEDICARE B/CHAMPUS': 'MEDICARE',
        
        'MEDICAID': 'MEDICAID',
        'MEDICAID/MGD CARE': 'MEDICARE_MEDICAID',
        'MEDICAID/MEDICARE': 'MEDICARE_MEDICAID',
        'CHAMPUS/MEDICAID': 'MEDICAID',
        'COMMERCIAL/MEDICAID': 'MEDICAID ADVANTAGE',
        'BLUE CROSS/MEDICAID': 'MEDICAID ADVANTAGE',
        'MISC/MEDICAID': 'MEDICAID',
    
        'SELF PAY': 'SELF PAY',
        'SELF-PAY': 'SELF PAY',
    
        'MANAGED CARE': 'MEDICARE ADVANTAGE',
        'MANAGED CARE/BCBS': 'MEDICARE ADVANTAGE',
        'MANAGED CARE/MISC': 'MEDICARE ADVANTAGE',
        'MANAGED CARE/COMM': 'MEDICARE ADVANTAGE',
        'MANAGED CARE/CHAMPUS': 'MEDICARE ADVANTAGE',
        'MISC/MANAGED CARE': 'MEDICARE ADVANTAGE',
    
        'BLUE CROSS': 'PRIVATE',
        'BLUE CROSS/COMM': 'PRIVATE',
        'BLUE CROSS/MEDICARE': 'MEDICARE ADVANTAGE',
        'BLUE CROSS/MGD CARE': 'MEDICARE ADVANTAGE',
        'BLUE CROSS/MISC': 'PRIVATE',
        'BLUE CROSS/BLUE CROS': 'PRIVATE',
        'BLUE CROSS/CHAMPUS': 'PRIVATE',
        'COMMERCIAL/BLUE CROS': 'PRIVATE',
    
        'COMMERCIAL': 'PRIVATE',
        'COMMERCIAL/MGD CARE': 'MEDICARE ADVANTAGE',
        'COMMERCIAL/MISC': 'PRIVATE',
        'COMMERCIAL/MEDICARE': 'MEDICARE ADVANTAGE',
        'COMMERCIAL/MEDICAID': 'MEDICAID ADVANTAGE',
        'COMMERCIAL/CHAMPUS': 'PRIVATE',
        'COMMERCIAL/COMM': 'PRIVATE',
    
        'CHAMPUS': 'GOV',
        'CHAMPUS/CHAMPVA': 'GOV',
        'CHAMPUS/MEDICAID': 'MEDICAID',
        'CHAMPUS/MEDICARE': 'MEDICARE',
        'CHAMPUS/COMMERCIAL': 'PRIVATE',
        'CHAMPUS/MISC': 'GOV',
        'CHAMPUS/MANAGED CARE': 'MEDICARE ADVANTAGE',
    
        'MGD CARE': 'MEDICARE ADVANTAGE',
        'MGD CARE/MEDICARE': 'MEDICARE ADVANTAGE',
        'MGD CARE/MEDICAID': 'MEDICARE_MEDICAID',
        'MGD CARE/BLUE CROSS': 'MEDICARE ADVANTAGE',
        'MGD CARE/CHAMPUS': 'MEDICARE ADVANTAGE',
        'MGD CARE/MGD CARE': 'MEDICARE ADVANTAGE',
    
        'GOVERNMENTAL': 'GOV',
        'GOVT/MISC': 'GOV',
    
        'WORKERS COMP': 'WORKERS COMP',
    
        'MISC': 'MISC',
        'MISC/MEDICARE': 'MEDICARE',
        'MISC/MEDICAID': 'MEDICAID',
        'MISC/BLUE CROSS': 'PRIVATE',
        'MISC/COMMERCIAL': 'PRIVATE',
        'MISC/CHAMPUS': 'GOV',
        'MISC/MISC': 'MISC',
    
        'PRUCARE': 'MISC'
    }
    
    admissions_df['insurance_cat'] = admissions_df['insurance'].map(ins_cleaning_dict)
    
    # drop na columns
    # admissions_df = admissions_df.dropna(axis=1)
    
    # race clean
    race_mapping = {
        '1': 'Others',
        '2': 'Others',
        '3': 'Others',
        'Alaskan Native': 'Others',
        'African Am.': 'African American  or Black',
        'Not Recorded': 'Unknown, Unavailable or Unreported',
        'CD:1594': 'Others',
        'Asian American': 'Multiple',
        'Patient Declines': 'Unknown, Unavailable or Unreported',
        'American Indian or Alaskan Native': 'Others',
        'Native Hawaiian or Other Pacific Islander': 'Others',
        'Hispanic': 'Others'
    }
    
    # Replace the values in the "race" column
    admissions_df["race"] = admissions_df["race"].replace(race_mapping)
    
    # keep only the most recent admittime to 
    admissions_df = admissions_df.loc[admissions_df.groupby('subject_id')['admittime'].idxmax()].copy()
    
    admissions_df['insurance_cat'].value_counts(normalize=True)
    
    # drop columns that contain all na
    admissions_df = admissions_df.dropna(axis=1, how='all')
    #print(head(admissions_df))
    print('admission table has been cleaned')
    return admissions_df

     

admissions_df = clean_admissions_table()

admission table has been cleaned


In [8]:
admissions_df

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_location,discharge_location,insurance,race,hospital_expire_flag,insurance_cat
0,10000100,58594204243,2019-06-26 13:20:00,2019-07-05 12:47:00,NaT,PHYSIC OFC/CLINIC,HOME SELF CARE,MEDICARE A,African American or Black,0,MEDICARE
1,10000562,60707883031,2016-06-02 09:24:00,2016-06-04 15:11:00,NaT,NON-HC FACILITY POO,HOME SELF CARE,MEDICAID,African American or Black,0,MEDICAID
2,10000671,67252991505,2019-05-14 20:20:00,2019-05-15 13:09:00,NaT,NON-HC FACILITY POO,HOME SELF CARE,SELF PAY,African American or Black,0,SELF PAY
7,10000683,51017647243,2015-11-10 14:37:00,2015-11-16 19:43:00,NaT,PHYSIC OFC/CLINIC,HOME SELF CARE,MANAGED CARE,Caucasian or White,0,MEDICARE ADVANTAGE
11,10000788,97722730874,2021-01-08 12:54:00,2021-01-21 16:50:00,NaT,NON-HC FACILITY POO,HOME HEALTH SERVICE,MEDICARE A,Caucasian or White,0,MEDICARE
...,...,...,...,...,...,...,...,...,...,...,...
972840,99999419,47502667961,2015-09-18 22:56:00,2015-09-21 13:47:00,NaT,NON-HC FACILITY POO,HOME SELF CARE,SELF-PAY,Caucasian or White,0,SELF PAY
972841,99999577,72376111354,2019-04-23 15:16:00,2019-04-24 19:51:00,NaT,TFR FROM OTHER HCF,HOME SELF CARE,MANAGED CARE,Caucasian or White,0,MEDICARE ADVANTAGE
972844,99999731,55258667078,2010-11-20 13:01:00,2010-11-21 18:45:00,NaT,NON-HC FACILITY POO,HOME SELF CARE,MANAGED CARE,African American or Black,0,MEDICARE ADVANTAGE
972845,99999856,39743800534,2019-09-21 08:20:00,2019-09-23 15:45:00,NaT,NON-HC FACILITY POO,HOME SELF CARE,MEDICARE,Caucasian or White,0,MEDICARE


In [9]:
merged_admissions_patients = pd.merge(df_patients, admissions_df, on='subject_id', how='inner')
merged_admissions_patients

,subject_id,gender,anchor_age,anchor_year,dod,hadm_id,admittime,dischtime,deathtime,admission_location,discharge_location,insurance,race,hospital_expire_flag,insurance_cat
0,10000100,Male,70,2019,None,58594204243,2019-06-26 13:20:00,2019-07-05 12:47:00,NaT,PHYSIC OFC/CLINIC,HOME SELF CARE,MEDICARE A,African American or Black,0,MEDICARE
1,10000562,Female,18,2016,None,60707883031,2016-06-02 09:24:00,2016-06-04 15:11:00,NaT,NON-HC FACILITY POO,HOME SELF CARE,MEDICAID,African American or Black,0,MEDICAID
2,10000671,Male,28,2019,None,67252991505,2019-05-14 20:20:00,2019-05-15 13:09:00,NaT,NON-HC FACILITY POO,HOME SELF CARE,SELF PAY,African American or Black,0,SELF PAY
3,10000683,Male,34,2015,None,51017647243,2015-11-10 14:37:00,2015-11-16 19:43:00,NaT,PHYSIC OFC/CLINIC,HOME SELF CARE,MANAGED CARE,Caucasian or White,0,MEDICARE ADVANTAGE
4,10000788,Female,83,2017,None,97722730874,2021-01-08 12:54:00,2021-01-21 16:50:00,NaT,NON-HC FACILITY POO,HOME HEALTH SERVICE,MEDICARE A,Caucasian or White,0,MEDICARE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492104,99999419,Male,47,2015,None,47502667961,2015-09-18 22:56:00,2015-09-21 13:47:00,NaT,NON-HC FACILITY POO,HOME SELF CARE,SELF-PAY,Caucasian or White,0,SELF PAY
492105,99999577,Male,60,2019,None,72376111354,2019-04-23 15:16:00,2019-04-24 19:51:00,NaT,TFR FROM OTHER HCF,HOME SELF CARE,MANAGED CARE,Caucasian or White,0,MEDICARE ADVANTAGE
492106,99999731,Male,38,2009,None,55258667078,2010-11-20 13:01:00,2010-11-21 18:45:00,NaT,NON-HC FACILITY POO,HOME SELF CARE,MANAGED CARE,African American or Black,0,MEDICARE ADVANTAGE
492107,99999856,Female,69,2019,None,39743800534,2019-09-21 08:20:00,2019-09-23 15:45:00,NaT,NON-HC FACILITY POO,HOME SELF CARE,MEDICARE,Caucasian or White,0,MEDICARE


## Removing non-important columns

In [10]:
merged_admissions_patients=merged_admissions_patients.drop(["anchor_year", "admittime", "dischtime", "deathtime", "discharge_location", "insurance"], axis=1)

In [11]:
merged_admissions_patients.drop_duplicates()

,subject_id,gender,anchor_age,dod,hadm_id,admission_location,race,hospital_expire_flag,insurance_cat
0,10000100,Male,70,None,58594204243,PHYSIC OFC/CLINIC,African American or Black,0,MEDICARE
1,10000562,Female,18,None,60707883031,NON-HC FACILITY POO,African American or Black,0,MEDICAID
2,10000671,Male,28,None,67252991505,NON-HC FACILITY POO,African American or Black,0,SELF PAY
3,10000683,Male,34,None,51017647243,PHYSIC OFC/CLINIC,Caucasian or White,0,MEDICARE ADVANTAGE
4,10000788,Female,83,None,97722730874,NON-HC FACILITY POO,Caucasian or White,0,MEDICARE
...,...,...,...,...,...,...,...,...,...
492104,99999419,Male,47,None,47502667961,NON-HC FACILITY POO,Caucasian or White,0,SELF PAY
492105,99999577,Male,60,None,72376111354,TFR FROM OTHER HCF,Caucasian or White,0,MEDICARE ADVANTAGE
492106,99999731,Male,38,None,55258667078,NON-HC FACILITY POO,African American or Black,0,MEDICARE ADVANTAGE
492107,99999856,Female,69,None,39743800534,NON-HC FACILITY POO,Caucasian or White,0,MEDICARE


In [12]:
merged_admissions_patients.to_csv("merged_admissions_patients.csv")